In [25]:
import pandas as pd
from tqdm import tqdm
import numpy as np

In [ ]:
#df= pd.read_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step1_5.csv')

df= pd.read_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step1_5.csv')

In [27]:
print(df['Muscle'].sum())
#,'Vocalization','Body Movements','Facial Expressions','CPOT(SUM)','CPOT(SUM)'

112


In [28]:
distinct_stay_id = df['stay_id'].unique()
print(f'總患者數:{len(distinct_stay_id)}')

總患者數:5870


In [29]:
"""
計算累積次數，如遇0超過兩次則重新計算
"""
def count_acc(data):
    count = 0
    count_0 = 0
    result = []
    started = False  # 加入標誌，表示是否開始計算

    for value in data:
        if value == 1:
            started = True  # 遇到第一個 1，開始計算
            count += 1
            count_0 = 0  # 遇到 1 時重置連續 0 的計數
        elif started:  # 只在開始計算後處理 0 的情況
            count_0 += 1
            if count_0 > 2:
                count = 0  # 超過兩個連續 0，計數器歸零
            else:
                count += 1  # 計算連續 0 的情況
        result.append(count if started else 0)  # 未開始計算前結果為 0

    return result

In [30]:
"""
衍生特徵
1. use_vent_consecutive_7days(連續呼吸器使用7天)
#棄用
2. use_vent_acc(累積呼吸次數)
"""
def add_use_vent_features(df_in):
    df_P = df_in.copy()
    df_P['use_vent_consecutive_7days'] = df_P['use_vent'].rolling(window=7).apply(lambda x: x.sum() >= 7).fillna(0)
    lase_vent_index = -1
    for i in range(len(df_P)):
        if df_P.at[i, 'use_vent'] == 1:
            # 检查上次的 dialysis 和当前之间是否有 <= 3 个 0
            if lase_vent_index != -1 and (i - lase_vent_index - 1) <= 3:
                for j in range(lase_vent_index + 1, i):
                    df_P.at[j, 'use_vent'] = 1
                 
    df_P['use_vent_acc'] = count_acc(df_P['use_vent'])
    return df_P

"""
衍生特徵
1. dialysis_over3(連續洗腎3天)
2. dialysis_over7(連續洗腎7天)
"""
def add_dialysis_features(df_in):
    df_P = df_in.copy()
    df_P['dialysis_over3'] = df_P['dialysis'].rolling(window=3).apply(lambda x: x.sum() >= 3).fillna(0)
    df_P['dialysis_over7'] = df_P['dialysis'].rolling(window=7).apply(lambda x: x.sum() >= 7).fillna(0)
    #if df_P['dialysis_over3'].max()>0:
    #    input(df_P[['dialysis','dialysis_over3']])
    return df_P

"""
衍生特徵
1. dialysis_since_last_event (距離上次洗腎經過的時間)
2. dialysis_acc(累積洗腎次數)
"""
def calculate_since_last_dialysis(df_in): 
    df_P = df_in.copy()
    df_P['dialysis_acc'] = df_P['dialysis'].cumsum()
    lase_dialysis_index = -1
    for i in range(len(df_P)):
        if df_P.at[i,'dialysis'] == 1:
            lase_dialysis_index = i
        
        if lase_dialysis_index == -1:
            df_P.at[i,'dialysis_since_last_event'] = 20
        else:
            df_P.at[i,'dialysis_since_last_event'] = i-lase_dialysis_index
    return df_P

"""
衍生特徵
#棄用
1.. vasopressor_acc(累積Vasopressor使用次數)
"""
# def add_use_Vasopressor_features(df_in):
#     df_P = df_in.copy()
#     lase_vasopressor_index = -1
#     for i in range(len(df_P)):
#         if df_P.at[i, 'Vasopressor'] == 1:
#             # 检查上次的 dialysis 和当前之间是否有 <= 2 个 0
#             if lase_vasopressor_index != -1 and (i - lase_vasopressor_index - 1) <= 2:
#                 for j in range(lase_vasopressor_index + 1, i):
#                     df_P.at[j, 'Vasopressor'] = 1
                    
#     df_P['use_vasopressor_acc'] = count_acc(df_P['Vasopressor'])
#     return df_P





'\n衍生特徵\n#棄用\n1.. vasopressor_acc(累積Vasopressor使用次數)\n'

In [31]:
df_result_list = []

for stay_id in tqdm(distinct_stay_id, desc="Processing stay_ids"):
    df_P = df[df['stay_id'] == stay_id]
    df_P.reset_index(drop=True, inplace=True)
    
    df_P = add_use_vent_features(df_P)
    df_P = add_dialysis_features(df_P)
    df_P = calculate_since_last_dialysis(df_P)
    #df_P = add_use_Vasopressor_features(df_P)
    df_result_list.append(df_P)

df_result = pd.concat(df_result_list, ignore_index=True)

Processing stay_ids: 100%|██████████| 5870/5870 [00:24<00:00, 243.13it/s]


In [ ]:
#df_result.to_csv(f'C:/Users/M1107171/MIMIC/清出來的資料/DNR/20240616/full_step1_6.csv',index = False)

df_result.to_csv(f'C:/Users/USER/M1326168/MIMIC/DNR/20241002/full_step1_6.csv',index = False)

In [33]:

"""
# 使用範例
df_P = pd.DataFrame({
    'use_vent': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
    'dialysis': [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]
})
df_P = add_use_vent_features(df_P)
df_P = calculate_since_last_dialysis(df_P)

print(df_P)
"""

"\n# 使用範例\ndf_P = pd.DataFrame({\n    'use_vent': [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],\n    'dialysis': [0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0]\n})\ndf_P = add_use_vent_features(df_P)\ndf_P = calculate_since_last_dialysis(df_P)\n\nprint(df_P)\n"

In [34]:
print(df_result['dialysis_over3'].sum())
print(df_result['dialysis_over7'].sum())

0.0
0.0


In [36]:
#print(df_result['use_vent_acc'].sum())
# 計算平均值
#mean_use_vent_acc = df['use_vent_acc'].mean()

# 計算四分位數
#quartiles_use_vent_acc = df['use_vent_acc'].quantile([0.25, 0.5, 0.75])

print(f"平均值: {df_result['use_vent_acc'].mean()}")
print(f"四分位數: \n{df_result['use_vent_acc'].quantile([0.25, 0.5, 0.75])}")
print(f"標準差: {df_result['use_vent_acc'].std()}")

平均值: 0.6286201022146508
四分位數: 
0.25    0.0
0.50    1.0
0.75    1.0
Name: use_vent_acc, dtype: float64
標準差: 0.48321490795506905
